In [3]:
import pygame
import random
import math
from collections import Counter

# Initialize pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("K-Nearest Neighbors Demo")

# Colors
RED = (255, 0, 0)
GREEN = (0, 200, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# Dot radius
RADIUS = 8

# Number of initial points per class
NUM_POINTS = 30

# K for KNN
K = 5

# Generate random points for red and green teams
red_points = [(random.randint(50, WIDTH-50), random.randint(50, HEIGHT-50)) for _ in range(NUM_POINTS)]
green_points = [(random.randint(50, WIDTH-50), random.randint(50, HEIGHT-50)) for _ in range(NUM_POINTS)]

# Combine points with labels
# Label 0 for red, 1 for green
data_points = [(p, 0) for p in red_points] + [(p, 1) for p in green_points]

def distance(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

def knn_classify(point, data, k=5):
    # Calculate distances to all points
    distances = []
    for data_point, label in data:
        dist = distance(point, data_point)
        distances.append((dist, label))
    # Sort by distance
    distances.sort(key=lambda x: x[0])
    # Take k nearest labels
    k_nearest_labels = [label for _, label in distances[:k]]
    # Majority vote
    vote = Counter(k_nearest_labels)
    return vote.most_common(1)[0][0]

def draw_dot(pos, team):
    # team 0 = red, 1 = green
    color = RED if team == 0 else GREEN
    pygame.draw.circle(screen, color, pos, RADIUS)

def draw_text(text, pos, color=BLACK, size=24):
    font = pygame.font.SysFont(None, size)
    img = font.render(text, True, color)
    screen.blit(img, pos)

def main():
    clock = pygame.time.Clock()
    running = True

    while running:
        screen.fill(WHITE)

        # Draw all existing points
        for p, label in data_points:
            draw_dot(p, label)

        # Get mouse position
        mouse_pos = pygame.mouse.get_pos()

        # Classify mouse position
        classification = knn_classify(mouse_pos, data_points, K)

        # Draw the new point with classification color but bigger radius
        color = RED if classification == 0 else GREEN
        pygame.draw.circle(screen, color, mouse_pos, RADIUS + 4, 3)  # outline circle
        pygame.draw.circle(screen, color, mouse_pos, RADIUS)

        # Display info text
        draw_text(f"K = {K}", (10, 10))
        team_name = "Red 🔴" if classification == 0 else "Green 🟢"
        draw_text(f"Classified as: {team_name}", (10, 40))

        # Event handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            # Optional: Add new point on mouse click
            if event.type == pygame.MOUSEBUTTONDOWN:
                # Add the new point with its classified label to the dataset
                data_points.append((mouse_pos, classification))

        pygame.display.flip()
        clock.tick(60)

    pygame.quit()

if __name__ == "__main__":
    main()